In [23]:
!pip install transformers tensorflow pandas scikit-learn

In [28]:
import pandas as pd
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import random

In [25]:
# Load IndoBERT model dan tokenizer
model_name = "indobenchmark/indobert-base-p2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

from google.colab import drive
drive.mount('/content/drive')

# Import library yang diperlukan
import pandas as pd

# Tentukan path lengkap ke file
file_path = '/content/drive/My Drive/Colab Notebooks/flashcard2.xlsx'

# Baca file Excel
data = pd.read_excel(file_path)

Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
data

,tema,pertanyaan,jawaban
0,Matematika,Berapakah hasil dari 7 + 5?,12
1,Matematika,Jika Ani memiliki 10 apel dan memberikan 4 kep...,6
2,Matematika,Sebutkan nama bangun datar yang memiliki 4 sis...,Persegi
3,Bahasa Indonesia,Sebutkan huruf vokal dalam alfabet!,"A, I, U, E, O"
4,Bahasa Indonesia,"Apa lawan kata dari ""besar""?",Kecil
5,Bahasa Indonesia,Sebutkan 3 nama hewan yang dimulai dengan huru...,"Kucing, Kuda, Kelinci"
6,IPA,Apa fungsi akar pada tumbuhan?,Menyerap air dan zat hara dari tanah
7,IPA,Sebutkan contoh hewan yang berkembang biak den...,"Ayam, Ikan, Burung"
8,IPA,Matahari terbit dari arah mana?,Timur
9,PKn,Apa bunyi sila pertama Pancasila?,Ketuhanan Yang Maha Esa


In [37]:
def get_sentence_embedding(sentence):
    # Tokenize input sentence and prepare it for model
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="tf")

    # Generate embeddings from IndoBERT model
    outputs = model(inputs.input_ids, attention_mask=inputs.attention_mask)

    # Use the last hidden state (token embeddings) to get a sentence embedding
    embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

    # Average token embeddings to get a fixed-size sentence embedding
    sentence_embedding = tf.reduce_mean(embeddings, axis=1)  # Shape: (batch_size, hidden_size)

    return sentence_embedding.numpy()  # Convert to numpy array for cosine similarity calculation

# Randomly select a question from the dataset
random_index = random.randint(0, len(data) - 1)
pertanyaan = data.loc[random_index, "pertanyaan"]
jawaban_dataset = data.loc[random_index, "jawaban"]

# Display the question and answer from the dataset
print(f"Pertanyaan: {pertanyaan}")

# Input from the user
user_input = input("Masukkan jawaban Anda: ")

# Get embeddings for the user's answer and dataset answer
user_embedding = get_sentence_embedding(user_input)
dataset_embedding = get_sentence_embedding(jawaban_dataset)

# Calculate cosine similarity
similarity = cosine_similarity(user_embedding, dataset_embedding)
print(f"Similarity Score: {similarity[0][0]}")

# Validation based on threshold
threshold = 0.85  # For example, minimal similarity of 85%
if similarity[0][0] >= threshold:
    print("Jawaban Anda valid!")
else:
    print("Jawaban Anda kurang sesuai.")

Pertanyaan: Apa arti peribahasa "Bagai air di daun talas"?
Masukkan jawaban Anda: waduh
Similarity Score: 0.6162815093994141
Jawaban Anda kurang sesuai.
